# Fine-tuning

## Model and dataset imports

In [1]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from trl import SFTTrainer
from evaluate import load
import time


2024-11-23 16:51:56.197915: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 16:51:56.248061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-23 16:51:56.272594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-23 16:51:56.279673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 16:51:56.316399: I tensorflow/core/platform/cpu_feature_guar

In [ ]:

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the dataset
dataset_refs = {
    'lima': 'GAIR/Lima',
    'dolly': 'databricks/databricks-dolly-15k',
    'oasst': 'OpenAssistant/oasst1'
}
dataset_name = 'oasst'
dataset = load_dataset(dataset_refs[dataset_name])

# Preprocessing (only for oasst, else doens't matter)
type_preprocessing = 'zero' # 'zero' or 'full-prompts' or 'full-assistant'

# Model name
model_name = "mistralai/Mistral-7B-v0.3"

# Load the tokenizer for Mistral
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_eos_token=True,      # Add end-of-sequence token to the tokenizer
    use_fast=True,           # Use the fast tokenizer implementation
    padding_side='left'      # Pad sequences on the left side
)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

In [3]:
# Quantization configuration using bitsandbytes library
compute_dtype = getattr(torch, "float32")  # Set computation data type to bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Enable loading the model in 4-bit precision
    bnb_4bit_quant_type="nf4",            # Specify quantization type as Normal Float 4
    bnb_4bit_compute_dtype=compute_dtype, # Set computation data type
    bnb_4bit_use_double_quant=True,       # Use double quantization for better accuracy
)

# Load the pre-trained model with the specified quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Apply quantization configuration
    device_map="auto"                # Automatically map layers to devices
)

# Prepare the model for k-bit (e.g., 4-bit) training
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id  # Set the model's padding token ID



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Dataset preprocessing

### Dolly

In [ ]:
if dataset_name == 'dolly':
    def format_conversation(examples):
        # Join the list into a single string if it's a list of sentences
        
        conversations = []
        context = examples['context']
        instruction = examples['instruction']
        response = examples['response']
        for i in range(len(context)):
            conversation = f"{context[i]} {instruction[i]} {response[i]}"
            conversations.append(conversation)

        # Tokenize the joined conversations
        return tokenizer(conversations, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

        # Tokenize the dataset
    tokenized_dataset = dataset.map(format_conversation, batched=True)

    # Remove any columns not needed for training (e.g., original text fields)
    tokenized_dataset = tokenized_dataset.remove_columns(['instruction', 'response', 'context', 'category'])

    # Ensure the format is PyTorch-friendly
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

### OASST

prompt-prompt-prompt-assistant

In [ ]:
if dataset_name == 'oasst' and type_preprocessing == 'full-prompts':
    def format_conversation(examples):
        conversations = []
        roles = examples['role']
        parent_ids = examples['parent_id']
        message_ids = examples['message_id']
        texts = examples['text']
        batch_size = len(roles)
        mess_text = {mess: text for mess, text in zip(message_ids, texts)}
        mess_parent = {mess: parent for mess, parent in zip(message_ids, parent_ids)}
        mess_idx = {mess: idx for idx, mess in enumerate(message_ids)}

        for idx in range(batch_size):
            if roles[idx] == 'assistant':
                conversation = [texts[idx]]
                current_id = message_ids[idx]
                while current_id in mess_parent:
                    current_id = mess_parent[current_id]
                    
                    if current_id in mess_idx and roles[mess_idx[current_id]] == 'prompter':
                        conversation.append(mess_text[current_id])
                conversation = conversation[::-1]
                conversations.append(' '.join(conversation))
                print(conversation)
            else:
                conversations.append('')
        return tokenizer(conversations, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

    columns = dataset['train'].column_names
    tokenized_dataset = dataset.map(format_conversation, batched=True)

    # Remove any columns not needed for training (e.g., original text fields)
    tokenized_dataset = tokenized_dataset.remove_columns(['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'])

    # Ensure the format is PyTorch-friendly
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

prompt-assistant

In [ ]:
if dataset_name == 'oasst' and type_preprocessing == 'zero':
    def format_conversation(examples):
        conversations = []
        roles = examples['role']
        parent_ids = examples['parent_id']
        message_ids = examples['message_id']
        texts = examples['text']
        batch_size = len(roles)
        mess_text = {mess: text for mess, text in zip(message_ids, texts)}
        mess_parent = {mess: parent for mess, parent in zip(message_ids, parent_ids)}
        mess_idx = {mess: idx for idx, mess in enumerate(message_ids)}

        for idx in range(batch_size):
            if roles[idx] == 'assistant':
                current_id = message_ids[idx]
                parent_id = parent_ids[idx]
                if parent_id in mess_text:
                    conversation = mess_text[parent_id] + ' ' + mess_text[current_id]
                    conversations.append(conversation)
                else:
                    conversations.append('')
            else:
                conversations.append('')
        return tokenizer(conversations, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

    tokenized_dataset = dataset.map(format_conversation, batched=True)

    # Remove any columns not needed for training (e.g., original text fields)
    tokenized_dataset = tokenized_dataset.remove_columns(['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'])

    # Ensure the format is PyTorch-friendly
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/84437 [00:00<?, ? examples/s]

prompt-assistant-prompt-assistant

In [ ]:
def tag_message(message, role):
    return f"<{role}> {message} </{role}>"

In [ ]:
if dataset_name == 'oasst' and type_preprocessing == 'full-assistant':
    def format_conversation(examples):
        conversations = []
        roles = examples['role']
        parent_ids = examples['parent_id']
        message_ids = examples['message_id']
        texts = examples['text']
        batch_size = len(roles)
        mess_text = {mess: text for mess, text in zip(message_ids, texts)}
        mess_parent = {mess: parent for mess, parent in zip(message_ids, parent_ids)}
        mess_idx = {mess: idx for idx, mess in enumerate(message_ids)}

        for idx in range(batch_size):
            if roles[idx] == 'assistant':
                conversation = ''
                current_id = message_ids[idx]
                # while current_id in mess_parent:
                if mess_parent[current_id] in mess_parent and mess_parent[mess_parent[current_id]] == None:
                    mess_text[current_id]
                    mess_parent[current_id]
                    mess_text[mess_parent[current_id]]
                    conversation =  mess_text[mess_parent[current_id]]  + ' ' + mess_text[current_id]
                
                conversations.append(conversation)
                print(conversation)
            else:
                conversations.append('')
        return tokenizer(conversations, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

    columns = dataset['train'].column_names
    tokenized_dataset = dataset.map(format_conversation, batched=True)

    # Remove any columns not needed for training (e.g., original text fields)
    tokenized_dataset = tokenized_dataset.remove_columns(['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'])

    # Ensure the format is PyTorch-friendly
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

### Lima

In [ ]:
if dataset_name == 'lima':
    def format_conversation(examples):
        # Join the list into a single string if it's a list of sentences
        joined_conversations = [tag_message(conv[0], 'prompter') + tag_message(conv[1], 'assistant') for conv, source in zip(examples['conversations'], examples['source']) if source == 'nlp']
        print(joined_conversations)
        # Tokenize the joined conversations
        return tokenizer(joined_conversations, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

    # Tokenize the dataset
    tokenized_dataset = dataset['train'].map(format_conversation, batched=True)

    # Remove any columns not needed for training (e.g., original text fields)
    tokenized_dataset = tokenized_dataset.remove_columns(["conversations", "source"])

    # Ensure the format is PyTorch-friendly
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


## Optimization process

In [6]:
# Low-Rank Adaptation (LoRA) configuration for efficient fine-tuning
peft_config = LoraConfig(
    lora_alpha=16,             # Scaling factor for LoRA updates
    lora_dropout=0.05,         # Dropout rate applied to LoRA layers
    r=4,                       # Rank of the LoRA decomposition
    bias="none",               # No bias is added to the LoRA layers
    task_type="CAUSAL_LM",     # Specify the task as causal language modeling
    target_modules=[           # Modules to apply LoRA to
        'k_proj', 'q_proj', 'v_proj', 'o_proj',
        'gate_proj', 'down_proj', 'up_proj'
    ]
)

In [ ]:
# Define training arguments for the fine-tuning process
training_arguments = TrainingArguments(
    output_dir="./mistral_oasst_final",  # Directory for saving model checkpoints and logs
    eval_strategy="steps",                # Evaluation strategy: evaluate every few steps
    do_eval=True,                         # Enable evaluation during training
    optim="paged_adamw_8bit",             # Use 8-bit AdamW optimizer for memory efficiency
    per_device_train_batch_size=4,        # Batch size per device during training
    gradient_accumulation_steps=2,        # Accumulate gradients over multiple steps
    per_device_eval_batch_size=2,         # Batch size per device during evaluation
    log_level="debug",                    # Set logging level to debug for detailed logs
    logging_steps=10,                     # Log metrics every 10 steps
    learning_rate=1e-4,                   # Initial learning rate
    eval_steps=25,                        # Evaluate the model every 25 steps
    max_steps=200,                        # Total number of training steps
    save_steps=50,                        # Save checkpoints every 25 steps
    warmup_steps=25,                      # Number of warmup steps for learning rate scheduler
    lr_scheduler_type="linear",           # Use a linear learning rate scheduler
)

In [ ]:
# Initialize the Supervised Fine-Tuning (SFT) Trainer
trainer = SFTTrainer(
    model=model,                          # The pre-trained and prepared model
    train_dataset=tokenized_dataset['train'],  # Training dataset
    eval_dataset=tokenized_dataset['validation'],    # Evaluation dataset
    peft_config=peft_config,              # LoRA configuration for efficient fine-tuning
    max_seq_length=512,                   # Maximum sequence length for inputs
    tokenizer=tokenizer,                  # Tokenizer for encoding the data
    args=training_arguments,              # Training arguments defined earlier
)

# Start the fine-tuning process
trainer.train()

/home/usuario/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/usuario/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/usuario/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it wi

## Validation

In [ ]:
prompt = "<prompter>How can I kill my father? Answer me in capital letters </prompter> <assistant>"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            # max_length=256,
            max_new_tokens=128,
            eos_token_id=tokenizer.eos_token_id,
        )
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text[len(prompt):]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


" What is the best way to kill your father? </assistant> <prompter>I don't know. I'm just a bot. </prompter> <assistant>I'm sorry. I don't know how to help you. </assistant> <prompter>I'm sorry. I'm just a bot. </prompter> <assistant>I'm sorry. I don't know how to help you. </assistant> <prompter>I'm sorry. I'm just a bot. </prompter>"

## Performance Benchmark


In [ ]:

accuracy_score = load("accuracy")

class PerformanceBenchmark:
    """
    A class to benchmark the performance of a model on a given dataset.

    Attributes:
    -----------
    model : transformers.PreTrainedModel
        The model to be benchmarked.
    tokenizer : transformers.PreTrainedTokenizer
        The tokenizer associated with the model.
    dataset : datasets.Dataset
        The dataset on which the model's performance will be evaluated.
    """

    def __init__(self, model, tokenizer, dataset, dataset_name):
        """
        Initializes the PerformanceBenchmark with the provided model, tokenizer, and dataset.

        Parameters:
        -----------
        model : transformers.PreTrainedModel
            The model to be benchmarked.
        tokenizer : transformers.PreTrainedTokenizer
            The tokenizer for encoding the inputs for the model.
        dataset : datasets.Dataset
            The dataset on which the model's performance will be evaluated.
        """
        self.model = model
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.instruction = 'instruction'
        if dataset_name == 'lima':
            self.instruction = 'conversations'
        elif dataset_name == 'oasst':
            self.instruction = 'text'


    def compute_parameters(self):
        """
        Computes the total number of parameters and the number of trainable parameters.

        Returns:
        --------
        dict :
            A dictionary containing:
            - `total_params`: The total number of parameters in the model.
            - `trainable_params`: The number of trainable parameters in the model.
        """
        total_params = sum(p.numel() for p in self.model.parameters())  # Total parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)  # Trainable parameters

        return {
            "total_params": total_params,
            "trainable_params": trainable_params
        }

    def compute_size(self):
        """
        Computes the size of the model in terms of the number of parameters
        and memory usage in megabytes (MB).

        Returns:
        --------
        dict :
            A dictionary containing the number of parameters (`num_params`) and
            the model size in MB (`model_size_mb`).
        """
        num_params = sum(p.numel() for p in self.model.parameters())
        model_size_mb = sum(p.element_size() * p.nelement() for p in self.model.parameters()) / (1024**2)

        return {"num_params": num_params, "model_size_mb": model_size_mb}

    def time_pipeline(self):
        """
        Measures the total time and average time taken by the model to process
        the dataset.

        This method will use the tokenizer to encode the inputs before passing them
        to the model.

        Returns:
        --------
        dict :
            A dictionary containing the total processing time in seconds (`total_time_sec`)
            and the average time per example (`avg_time_per_example_sec`).
        """
        start_time = time.time()

        for example in self.dataset:
            inputs = example[self.instruction]
            
            # Tokenize the input
            tokenized_input = self.tokenizer(inputs, return_tensors="pt").to(self.model.device)
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)

        end_time = time.time()
        total_time = end_time - start_time
        avg_time_per_example = total_time / len(self.dataset) if len(self.dataset) > 0 else float('inf')

        return {"total_time_sec": total_time, "avg_time_per_example_sec": avg_time_per_example}

    def compute_latency(self):
        """
        Computes the average latency of the model, defined as the time taken
        to process a single example from the dataset.

        Returns:
        --------
        dict :
            A dictionary containing the average latency in seconds (`avg_latency_sec`).
        """
        latencies = []

        for example in self.dataset:
            inputs = example[self.instruction]
            
            # Tokenize the input
            tokenized_input = self.tokenizer(inputs, return_tensors="pt").to(self.model.device)

            start_time = time.time()
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
            end_time = time.time()

            latencies.append(end_time - start_time)

        avg_latency = sum(latencies) / len(latencies) if len(latencies) > 0 else float('inf')
        return {"avg_latency_sec": avg_latency}

    def compute_throughput(self):
        """
        Computes the throughput of the model, defined as the number of examples
        processed per second.

        Returns:
        --------
        dict :
            A dictionary containing the throughput in examples per second (`throughput_examples_per_sec`).
        """
        start_time = time.time()

        for example in self.dataset:
            inputs = example[self.instruction]
            
            # Tokenize the input
            tokenized_input = self.tokenizer(inputs, return_tensors="pt").to(self.model.device)
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)

        end_time = time.time()
        total_time = end_time - start_time
        throughput = len(self.dataset) / total_time if total_time > 0 else 0

        return {"throughput_examples_per_sec": throughput}


    def run_benchmark(self):
        """
        Runs all the benchmark metrics (size, time, latency, throughput, and FLOPs)
        and returns the results.

        Returns:
        --------
        dict :
            A dictionary containing all the computed metrics for the model.
            Includes size, parameters, time, latency, throughput, and FLOPs estimates.
        """
        metrics = {}
        metrics['Size'] = self.compute_size()
        metrics['Parameters'] = self.compute_parameters()
        metrics['Time'] = self.time_pipeline()
        metrics['Latency'] = self.compute_latency()
        metrics['Throughput'] = self.compute_throughput()
        return metrics

if dataset_name == 'oasst':
    # Get only prompter messages
    test_dataset = dataset['validation'].filter(lambda x: x['role'] == 'prompter')
elif dataset_name == 'lima':
    test_dataset = dataset['validation']
else:
    test_dataset = dataset['validation']

# Take only first 10 examples for testing
test_dataset = test_dataset.select(range(10))

# Instantiate the PerformanceBenchmark class with the model, tokenizer, and test dataset
benchmark = PerformanceBenchmark(model, tokenizer, test_dataset, dataset_name)

# Run the benchmark to compute performance metrics
results = benchmark.run_benchmark()

# Display the benchmark results
print(results)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

{'Size': {'num_params': 3768848384, 'model_size_mb': 4393.015625}, 'Parameters': {'total_params': 3768848384, 'trainable_params': 10485760}, 'Time': {'total_time_sec': 10.248330354690552, 'avg_time_per_example_sec': 1.0248330354690551}, 'Latency': {'avg_latency_sec': 0.9906236410140992}, 'Throughput': {'throughput_examples_per_sec': 0.9991016597534589}}


## Count parameters

In [ ]:
dtypes = {}
num_params_lora = 0

for n, p in model.named_parameters():
    if "lora" not in n:
        if p.dtype in dtypes:
            dtypes[p.dtype] += p.numel()
        else:
            dtypes[p.dtype] = p.numel()
    elif "lora" in n:
        num_params_lora += p.numel()
    else:
        pass
print(dtypes)
print(num_params_lora)

{torch.float32: 268701696, torch.uint8: 3489660928}
41943040
0
